In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://security.ubuntu.com/ubuntu 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [4]:
# Create the vine_table DataFrame
# vine_df = df.select([])
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     

In [5]:
# Create a DataFrame or table where there are 20 or more total votes
filtered_df = vine_df.filter(vine_df.total_votes>=20)
filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N|
|R3SJYLG07EHNE9|          2|           16|         34|   N|                N|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
|R1KPOIT5WT3NEF|          1|           13|         29|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
| R33IFUWCNYFSO|          2|            4|         45|   N|                N|
| RWMT2MXG3WLDP|          5|            6|         20|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|     

In [6]:
# Create a DataFrame or table where the percentage of helpful_votes is equal to or greater than 50%
helpful_votes_df = filtered_df.filter(df.helpful_votes/df.total_votes>=.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|
|R3THZWOBS1086J|          5|           46|         47|   N|                N|
|R160DHJPJ4B20R|          4|           28|         30|   N|     

In [7]:
# Create a DataFrame or table where there is a Vine review
paid_vine_df=helpful_votes_df.filter(helpful_votes_df["vine"]=="Y")
paid_vine_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [8]:
# Create a DataFrame or table where there isn't a Vine review
unpaid_vine_df=helpful_votes_df.filter(helpful_votes_df["vine"]=="N")
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|
|R3THZWOBS1086J|          5|           46|         47|   N|                N|
|R160DHJPJ4B20R|          4|           28|         30|   N|     

In [9]:
# Determine the total number of reviews, the number of 5-star reviews, 
#and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
total_paid_review = paid_vine_df.count()
total_paid_review

0

In [10]:
total_unpaid_review = unpaid_vine_df.count()
total_unpaid_review

403807

In [11]:
total_review_count = total_paid_review + total_unpaid_review
total_review_count

403807

In [17]:
# Number of 5-star reviews
paid_5_star_reviews = paid_vine_df.filter(paid_vine_df["star_rating"] == "5").count()
paid_5_star_reviews 

0

In [18]:
unpaid_5_star_reviews = unpaid_vine_df.filter(unpaid_vine_df["star_rating"] == "5").count()
unpaid_5_star_reviews 

242889

In [20]:
total_5_star_reviews = paid_5_star_reviews + unpaid_5_star_reviews
total_5_star_reviews

242889

In [21]:
# Percentage of 5-star reviews
paid_5_star_percentage = round((paid_5_star_reviews/total_paid_review) * 100, 2)
paid_5_star_percentage

ZeroDivisionError: ignored

In [22]:
unpaid_5_star_percentage = round((unpaid_5_star_reviews/total_unpaid_review) * 100, 2)
unpaid_5_star_percentage

60.15